In [38]:
import re
import pandas as pd
import numpy as np
import os

In [39]:
# Set the directory path and file name
dir_path = os.path.abspath("../input/")
file1 = "usedcarsni_2023-02-22T21-43-40.csv"
file2 = "usedcarsni_2022-10-06T21-13-05.csv"

# Create the absolute file path
file1_path = os.path.join(dir_path, file1)
file2_path = os.path.join(dir_path, file2)

# Read the CSV file into a pandas DataFrame
df1 = pd.read_csv(file1_path)
df2 = pd.read_csv(file2_path)

# Set output folder path
output_path = os.path.abspath("../output/")


In [40]:
def extract_number(link):
    pattern = r'(\d{9})'  # This pattern matches a sequence of 9 digits.
    match = re.search(pattern, link)
    if match:
        return match.group(1)
    else:
        return None

def extract_unoque_string(link):
    unique_string = link.split("/")[-1].split("?")[0]
    return unique_string

In [41]:
#apply extract_number function to the link column
df1['ad_id'] = df1['link'].apply(extract_number)
df2['ad_id'] = df2['link'].apply(extract_number)

#apply extract_unoque_string function to the link column
df1['unique_string'] = df1['link'].apply(extract_unoque_string)
df2['unique_string'] = df2['link'].apply(extract_unoque_string)

In [42]:
#save the new dataframes to csv files adding the suffix _clean
df1.to_csv(file1.replace('.csv', '_clean.csv'), index=False)
df2.to_csv(file2.replace('.csv', '_clean.csv'), index=False)

section to get only unique rows

In [43]:
filea = file1.replace('.csv', '_clean.csv')
fileb = file2.replace('.csv', '_clean.csv')

def check_csv_file2(file1, file2):
    # Read in the two CSV files as DataFrames
    df1 = pd.read_csv(file1)
    df2 = pd.read_csv(file2)

    lista = df1['ad_id'].tolist()
    listb = df2['ad_id'].tolist()

    unique_values = set(lista) - set(listb)
    return unique_values


In [44]:
output = check_csv_file2(filea, fileb)

In [45]:
print(len(output))

15651


In [46]:
#save output to csv file to output folder
output_df = pd.DataFrame(output)
output_df.to_csv(os.path.join(output_path, 'unique_Ids_output.csv'), index=False)